# Forest Combustion Recognition

### Importing necessary libraries

In [1]:
from tensorflow.keras.models import Sequential     #used to initialize the model 
from tensorflow.keras.layers import Dense          #used for adding (building) layers
from tensorflow.keras.layers import Conv2D         #convolution layer
from tensorflow.keras.layers import MaxPool2D      #max pooling layer
from tensorflow.keras.layers import Flatten        #flatten layer

### Data pre-processing

In [2]:
from keras.preprocessing.image import ImageDataGenerator   #used for passing the images

Using TensorFlow backend.


In [3]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,horizontal_flip=True,zoom_range=0.2) 
test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
x_train = train_datagen.flow_from_directory(r'C:\Users\user\Internship\forest\train_data',target_size=(64,64),batch_size=32,class_mode='categorical')
x_test = test_datagen.flow_from_directory(r'C:\Users\user\Internship\forest\test_data',target_size=(64,64),batch_size=32,class_mode='categorical')


Found 1043 images belonging to 2 classes.
Found 261 images belonging to 2 classes.


### Adding layers

In [5]:
model = Sequential()

In [6]:
model.add(Conv2D(32,4,4,input_shape=(64,64,3),activation='relu'))

#32 - number of feature detector, 4,4- size
#input_shape - expected input shape (64,64) and 3- RGB  (1-Grey Scale)

In [7]:
model.add(MaxPool2D(pool_size=(2,2)))

In [8]:
model.add(Flatten())

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 32)        1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
Total params: 1,568
Trainable params: 1,568
Non-trainable params: 0
_________________________________________________________________


### Adding Dense Layer

In [10]:
model.add(Dense(units=128,activation='relu',kernel_initializer='random_uniform'))

In [11]:
model.add(Dense(units=2,kernel_initializer='random_uniform',activation='softmax'))

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 16, 32)        1568      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 8, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 128)               262272    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 258       
Total params: 264,098
Trainable params: 264,098
Non-trainable params: 0
_________________________________________________________________


In [13]:
model.compile(optimizer = 'adam',loss='categorical_crossentropy',metrics=['accuracy']) 

In [14]:
print(x_train.class_indices)   #to check what's 0 and 1
                               #dimention of x_train is 4== (number of images, height,width,rgb)

{'fire': 0, 'no_fire': 1}


### Training the model

In [15]:
model.fit_generator(x_train,steps_per_epoch=1043,validation_data=x_test,epochs=10,validation_steps=261)

Epoch 1/10
1043/1043 [==============================] - 110s 105ms/step - loss: 0.2091 - accuracy: 0.9138 - val_loss: 0.1896 - val_accuracy: 0.9272
Epoch 2/10
1043/1043 [==============================] - 114s 109ms/step - loss: 0.1160 - accuracy: 0.9523 - val_loss: 0.3379 - val_accuracy: 0.8889
Epoch 3/10
1043/1043 [==============================] - 121s 116ms/step - loss: 0.0675 - accuracy: 0.9734 - val_loss: 0.1881 - val_accuracy: 0.9310
Epoch 4/10
1043/1043 [==============================] - 130s 125ms/step - loss: 0.0441 - accuracy: 0.9831 - val_loss: 0.2465 - val_accuracy: 0.9272
Epoch 5/10
1043/1043 [==============================] - 117s 112ms/step - loss: 0.0360 - accuracy: 0.9863 - val_loss: 0.3230 - val_accuracy: 0.9119
Epoch 6/10
1043/1043 [==============================] - 117s 112ms/step - loss: 0.0358 - accuracy: 0.9863 - val_loss: 0.3210 - val_accuracy: 0.9080
Epoch 7/10
1043/1043 [==============================] - 126s 121ms/step - loss: 0.0226 - accuracy: 0.9915 - val_

In [16]:
model.save('forest.h5')      #saving the model

### Prediction

In [17]:
from tensorflow.keras.models import load_model  
import numpy as np
import cv2

In [18]:
model = load_model('forest.h5')
model.compile(optimizer = 'adam',loss='categorical_crossentropy',metrics=['accuracy']) 

In [19]:
from skimage.transform import resize

In [20]:
def detect(frame):
    try:
        img = resize(frame,(64,64))
        img = np.expand_dims(img,axis=0)
        if(np.max(img)>1):
            img = img/255.0
        prediction = model.predict(img)    
        print(prediction)
        prediction_class = model.predict_classes(img)
        print(prediction_class)   #gives class name
    except AttributeError:
        print("Shape not found")

In [21]:
frame = cv2.imread(r'C:\Users\user\Internship\forest\test_data\no_fire\image3.jpeg') #to read image
data = detect(frame)

[[6.685167e-22 1.000000e+00]]
[1]


In [22]:
frame = cv2.imread(r'C:\Users\user\Internship\forest\test_data\fire\images439.jpg') #to read image
data = detect(frame)

[[0.83967686 0.16032311]]
[0]
